In [1]:
!pip install -U transformers

     |████████████████████████████████| 769 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 4.0 MB/s eta 0:00:01
ERROR: allennlp 1.0.0 has requirement transformers<2.12,>=2.9, but you'll have transformers 3.0.2 which is incompatible.
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0
  Attempting uninstall: transformers
    Found existing installation: transformers 2.11.0
    Uninstalling transformers-2.11.0:
      Successfully uninstalled transformers-2.11.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

from transformers import BertTokenizer, BertPreTrainedModel, BertConfig, BertModel
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [3]:
random.seed(2020)
np.random.seed(2020)
torch.manual_seed(2020)
torch.cuda.manual_seed_all(2020)

In [4]:
def load_json_as_pandas_df(filename):
    df = pd.read_json(f'/kaggle/input/squad-20/{filename}', orient='records')
    df = pd.DataFrame.from_records(df['data'])
    df = df.explode('paragraphs').reset_index(drop=True)
    df['context'] = df['paragraphs'].apply(lambda x: x['context'])
    df['qas'] = df['paragraphs'].apply(lambda x: x['qas'])
    df = df[['title', 'context', 'qas']]
    df = df.explode('qas').reset_index(drop=True)
    df = pd.concat([df, pd.DataFrame.from_records(df['qas'])], axis=1).drop(['qas'], axis=1)
    df.loc[df['is_impossible'], 'answers'] = df.loc[df['is_impossible'], 'plausible_answers']
    df = df.drop(['plausible_answers'], axis=1)
    df['answers'] = df['answers'].apply(lambda x: x[0] if len(x) else {})
    return df

In [5]:
train_df = load_json_as_pandas_df('train-v2.0.json')
train_df

,title,context,question,id,answers,is_impossible
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,"{'text': 'in the late 1990s', 'answer_start': ...",False
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,"{'text': 'singing and dancing', 'answer_start'...",False
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,"{'text': '2003', 'answer_start': 526}",False
3,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,"{'text': 'Houston, Texas', 'answer_start': 166}",False
4,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,"{'text': 'late 1990s', 'answer_start': 276}",False
...,...,...,...,...,...,...
130314,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,5a7e070b70df9f001a875439,"{'text': 'matter', 'answer_start': 485}",True
130315,Matter,"The term ""matter"" is used throughout physics i...",Who coined the term partonic matter?,5a7e070b70df9f001a87543a,"{'text': 'Alfvén', 'answer_start': 327}",True
130316,Matter,"The term ""matter"" is used throughout physics i...",What is another name for anti-matter?,5a7e070b70df9f001a87543b,"{'text': 'Gk. common matter', 'answer_start': ...",True
130317,Matter,"The term ""matter"" is used throughout physics i...",Matter usually does not need to be used in con...,5a7e070b70df9f001a87543c,"{'text': 'a specifying modifier', 'answer_star...",True


In [6]:
valid_df = load_json_as_pandas_df('dev-v2.0.json')
valid_df

,title,context,question,id,answers,is_impossible
0,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,56ddde6b9a695914005b9628,"{'text': 'France', 'answer_start': 159}",False
1,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,56ddde6b9a695914005b9629,"{'text': '10th and 11th centuries', 'answer_st...",False
2,Normans,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,56ddde6b9a695914005b962a,"{'text': 'Denmark, Iceland and Norway', 'answe...",False
3,Normans,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,56ddde6b9a695914005b962b,"{'text': 'Rollo', 'answer_start': 308}",False
4,Normans,The Normans (Norman: Nourmands; French: Norman...,What century did the Normans first gain their ...,56ddde6b9a695914005b962c,"{'text': '10th century', 'answer_start': 671}",False
...,...,...,...,...,...,...
11868,Force,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,5737aafd1c456719005744ff,"{'text': 'sthène', 'answer_start': 665}",False
11869,Force,"The pound-force has a metric counterpart, less...",What does not have a metric counterpart?,5ad28ad0d7d075001a4299cc,"{'text': 'pound-force', 'answer_start': 4}",True
11870,Force,"The pound-force has a metric counterpart, less...",What is the force exerted by standard gravity ...,5ad28ad0d7d075001a4299cd,"{'text': 'kilogram-force', 'answer_start': 82}",True
11871,Force,"The pound-force has a metric counterpart, less...",What force leads to a commonly used unit of mass?,5ad28ad0d7d075001a4299ce,"{'text': 'kilogram-force', 'answer_start': 195}",True


In [7]:
max_len = 512
epochs = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
# question + context tokens length

# count    130319.000000
# mean        170.726632
# std          65.407215
# min          35.000000
# 25%         129.000000
# 50%         158.000000
# 75%         200.000000
# max         870.000000
# dtype: float64

# Around 200 examples have token length of more than 500

In [9]:
class SQuAD2Dataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, idx):
        context = self.df['context'][idx]
        question = self.df['question'][idx]
        answer = self.df['answers'][idx]['text']
        answer_start = self.df['answers'][idx]['answer_start']
        is_impossible = self.df['is_impossible'][idx]
        qas_id = self.df['id'][idx]
        
        input_text = '[CLS] ' + question + ' [SEP] ' + context + ' [SEP]'
        input_tokens = self.tokenizer.tokenize(input_text)
        
        start_position = 0
        end_position = 0
        
        if not is_impossible:
            answer_tokens = self.tokenizer.tokenize(answer)
            
            for i, tok in enumerate(input_tokens):
                if tok == answer_tokens[0] and input_tokens[min(len(input_tokens)-1, i+len(answer_tokens)-1)] == answer_tokens[-1]:
                    start_position = i
                    end_position = min(len(input_tokens)-1, i+len(answer_tokens)-1)

        token_type_ids = [0] * (len(self.tokenizer.tokenize(question)) + 2)
        token_type_ids = token_type_ids + [1] * (len(input_tokens) - len(token_type_ids))

        input_ids = self.tokenizer.convert_tokens_to_ids(input_tokens)
        attention_mask = [1] * len(input_tokens)

        # Truncate
        if len(input_ids) > self.max_len:
            input_ids = input_ids[:self.max_len]
            attention_mask = attention_mask[:self.max_len]
            token_type_ids = token_type_ids[:self.max_len]
            
        # Padding
        if len(input_ids) < self.max_len:
            input_ids = input_ids + [0] * (max_len - len(input_tokens))
            attention_mask = attention_mask + [0] * (max_len - len(input_tokens))
            token_type_ids = token_type_ids + [0] * (max_len - len(input_tokens))
        
        # convert lists to tensor
        input_ids = torch.tensor(input_ids, dtype=torch.long)
        attention_mask = torch.tensor(attention_mask, dtype=torch.float)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.long)
        start_position = torch.tensor(start_position, dtype=torch.long).unsqueeze(0)
        end_position = torch.tensor(end_position, dtype=torch.long).unsqueeze(0)
        
        return input_ids, attention_mask, token_type_ids, start_position, end_position

In [10]:
train_dataset = SQuAD2Dataset(train_df, tokenizer, max_len)
valid_dataset = SQuAD2Dataset(valid_df, tokenizer, max_len)

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

In [12]:
class SQuAD2Model(BertPreTrainedModel):
    def __init__(self, conf):
        super(SQuAD2Model, self).__init__(conf)
        self.bert = BertModel(conf)
        self.drop_out = nn.Dropout(0.1)
        self.fc = nn.Linear(768 * 2, 2)
    
    def forward(self, ids, mask, token_type_ids):
        _, _, out = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)

        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        logits = self.fc(out)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [ ]:
model_config = BertConfig.from_pretrained('bert-base-uncased')
model_config.output_hidden_states = True
model = SQuAD2Model(conf=model_config)
model.to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in tqdm(range(epochs)):

    model.train()

    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()

        input_ids, attention_masks, token_type_ids, start_positions, end_positions = batch
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        token_type_ids = token_type_ids.to(device)
        start_positions = start_positions.squeeze().to(device)
        end_positions = end_positions.squeeze().to(device)

        outputs_start, outputs_end = model(input_ids, attention_masks, token_type_ids)

        loss = criterion(outputs_start, start_positions) + criterion(outputs_end, end_positions)
        loss.backward()

        optimizer.step()
        if (step+1) % 2500 == 0:
            print(f'Epoch: {epoch+1} || Step: {step+1} || Training Loss: {loss.item()}')

    model.eval()

    true_start = []
    true_end = []
    pred_start = []
    pred_end = []

    with torch.no_grad():

        for step, batch in enumerate(valid_dataloader):

            input_ids, attention_masks, token_type_ids, start_positions, end_positions = batch
            input_ids = input_ids.to(device)
            attention_masks = attention_masks.to(device)
            token_type_ids = token_type_ids.to(device)
            start_positions = start_positions.squeeze().to(device)
            end_positions = end_positions.squeeze().to(device)

            outputs_start, outputs_end = model(input_ids, attention_masks, token_type_ids)

            true_start.append(start_positions)
            true_end.append(end_positions)
            pred_start.append(outputs_start)
            pred_end.append(outputs_end)

    true_start = torch.cat(true_start)
    true_end = torch.cat(true_end)
    pred_start = torch.cat(pred_start)
    pred_end = torch.cat(pred_end)
    
    loss = criterion(pred_start, true_start) + criterion(pred_end, true_end)
        
    print(f'Validation Loss: {loss.item()}')